# Set up

In [3]:
import os
import sys
#set custom path
sys.path.insert(0, os.path.abspath(r'\users\fynn\documents\anaconda\envs\tf_keras_gpu_test\lib\site-packages'))

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers
#for data encoding
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [5]:
#load data
data = pd.read_csv('data/testtrain.csv') #each row is color name, then RGB values (4 columns)

y = data.y
X = np.asarray(data.drop('y', axis = 1))

#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size=0.8, random_state = 0)
X_train.shape

(216000, 12)

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
#translate class labels
seq_classes = list(range(len(y_train.unique())))
seq_y_train = y_train.replace(y_train.unique(), seq_classes)
seq_y_test = y_test.replace(y_test.unique(), seq_classes)

In [7]:
seq_y_train

195713     0
101137     1
54696      2
74083      3
90035      1
          ..
164709     6
33793     12
40377      7
202291    13
86389      3
Name: y, Length: 216000, dtype: int64

# Model

In [11]:
#to ensure our model does not interpret int classes as regression we encode target as categories
train_labels = to_categorical(seq_y_train, num_classes = 15)
test_labels = to_categorical(seq_y_test, num_classes = 15)

In [12]:
train_labels[:5]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [15]:
#call basic feed forward NN
network = models.Sequential()
#build layers
#input layers (unit, activation, input)
network.add(layers.Dense(24, activation = 'relu', input_shape = (12*1,)))
#hidden layer [try without input_shape to see if it works]
network.add(layers.Dense(784, activation = 'relu'))
network.add(layers.Dropout(0.2))
#output layer
network.add(layers.Dense(15, activation = 'softmax'))


opt = keras.optimizers.Adam(learning_rate=0.01)
#compile model to configure for training
network.compile(optimizer=opt, #optomizers are basically different versions of Stochastic GD
                loss = 'categorical_crossentropy',
                metrics=['categorical_accuracy'])

In [16]:
#fit to train data
network.fit(X_train, train_labels, epochs=20, batch_size = 100, validation_split = 0.2)

Epoch 1/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.5261 - categorical_accuracy: 0.4332 - val_loss: 1.3805 - val_categorical_accuracy: 0.4829
Epoch 2/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.3851 - categorical_accuracy: 0.4796 - val_loss: 1.3366 - val_categorical_accuracy: 0.4934
Epoch 3/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.3401 - categorical_accuracy: 0.5006 - val_loss: 1.3016 - val_categorical_accuracy: 0.5072
Epoch 4/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.3230 - categorical_accuracy: 0.5068 - val_loss: 1.2947 - val_categorical_accuracy: 0.5201
Epoch 5/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.3043 - categorical_accuracy: 0.5144 - val_loss: 1.2683 - val_categorical_accuracy: 0.5225
Epoch 6/20
1728/1728 [==============================] - 2s 1ms/step - loss: 1.2974 - categorical_accuracy: 0.5173 - val_loss: 1.2676 - val_categorical_accuracy:

In [17]:
network_prob = network.predict(X_test)
network_pred = network_prob.argmax(axis=-1)
print(network_pred[:5], [seq_y_test[:5].values])
print(classification_report(seq_y_test, network_pred))

[2 8 9 5 8] [array([0, 1, 2, 3, 1], dtype=int64)]
              precision    recall  f1-score   support

           0       0.02      0.03      0.02      3600
           1       0.00      0.00      0.00      3600
           2       0.01      0.01      0.01      3600
           3       0.00      0.00      0.00      3600
           4       0.00      0.00      0.00      3600
           5       0.00      0.00      0.00      3600
           6       0.00      0.00      0.00      3600
           7       0.02      0.03      0.03      3600
           8       0.00      0.00      0.00      3600
           9       0.00      0.00      0.00      3600
          10       0.01      0.01      0.01      3600
          11       0.05      0.05      0.05      3600
          12       0.03      0.03      0.03      3600
          13       0.00      0.00      0.00      3600
          14       0.01      0.00      0.01      3600

    accuracy                           0.01     54000
   macro avg       0.01      0

# Convolutional